In [1]:
!nvcc --version
!nvidia-smi
import torch
torch.__version__

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri May 10 05:34:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8               9W /  7

'2.2.1+cu121'

In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-6o3we4ro
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-6o3we4ro
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.4 MB/s eta 0

In [3]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout solov2
!git pull origin solov2
%cd /content/TACO-expl/AdelaiDet/
!python setup.py build develop

/content
Cloning into 'TACO-expl'...
remote: Enumerating objects: 2308, done.
remote: Counting objects: 100% (2308/2308), done.
remote: Compressing objects: 100% (1105/1105), done.
remote: Total 2308 (delta 1207), reused 2237 (delta 1164), pack-reused 0
Receiving objects: 100% (2308/2308), 130.39 MiB | 20.09 MiB/s, done.
Resolving deltas: 100% (1207/1207), done.
/content/TACO-expl
Branch 'solov2' set up to track remote branch 'solov2' from 'origin'.
Switched to a new branch 'solov2'
From https://github.com/DomMcOyle/TACO-expl
 * branch            solov2     -> FETCH_HEAD
Already up to date.
/content/TACO-expl/AdelaiDet
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/adet
copying adet/__init__.py -> build/lib.linux-x86_64-cpython-310/adet
creating build/lib.linux-x86_64-cpython-310/adet/data
copying adet/data/__init__.py -> build/lib.linux-x86_64-cpython-310/adet/data
copying adet/data/dataset_mapper.p

In [4]:
!pip install rapidfuzz==2.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.1 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [2]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.modeling import build_model
%cd /content/TACO-expl/AdelaiDet/adet/modeling/solov2/
from solov2 import SOLOv2

/content
/content/TACO-expl/AdelaiDet/adet/modeling/solov2


In [3]:
import sys
#sys.path.append("/content/TACO-expl/MaskDINO")
#!cd /content/TACO-expl/MaskDINO/
#cfg_biggest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/swin/maskdino_R50_bs16_50ep_4s_dowsample1_2048.yaml'
#model_biggest_weights_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth'

#cfg_smallest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/maskdino_R50_bs16_50ep_3s.yaml'
#model_smallest_weight_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth'

cfg_solo_base_path = '/content/TACO-expl/AdelaiDet/configs/SOLOv2/Base-SOLOv2.yaml'
cfg_solo_r50_path = '/content/TACO-expl/AdelaiDet/configs/SOLOv2/R50_3x.yaml'

In [4]:
sys.path.append('/content/TACO-expl/AdelaiDet/adet/config')
from defaults import _C

def setup_cfg(cfg_base, cfg_backbone):
  cfg = _C
  cfg.merge_from_file(cfg_base)
  cfg.merge_from_file(cfg_backbone)
  return cfg

cfg_solov2 = setup_cfg(cfg_solo_base_path, cfg_solo_r50_path)
print(cfg_solov2)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('coco_2017_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    CROP_INSTANCE: True
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  HFLIP_TRAIN: True
  IS_ROTATE: False
  MASK_FORMAT: bitmask
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32, 64, 128, 256, 512]]
  BACKBONE:
    ANTI_ALIAS: False
    FREEZE_AT: 2
    NAME: build_resnet_fpn

In [5]:
model = build_model(cfg_solov2)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

46317392


In [4]:
%cd /content/TACO-expl
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask


from torchvision.ops import RoIAlign
from HDDETR.datasets.torchvision_datasets.coco import CocoDetection as TvCocoDetection # from torchvision.datasets import CocoDetection as TvCocoDetection
                                                                                       # the right above import import does not work bc of attribute definitions
import HDDETR.datasets.transforms as T #from torchvision import transforms as T # this import does not contain the randomselect method
import HDDETR.util.misc as mutils

/content/TACO-expl


In [5]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [6]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [7]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [8]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [9]:
#!mkdir /content/official/
!cp -r /content/MyDrive/MyDrive/official/ /content/
!mkdir /content/output/
!cp -r /content/MyDrive/MyDrive/solo_models/chkpt/ /content/output/

In [9]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_val.json'

img_dir = '/content/official/'

#register_coco_instances("TACO_train", {}, train_dataset.annotation_file, train_dataset.image_dir)
#MetadataCatalog.get("TACO_train").set(thing_classes=train_dataset.classes)
#dataset_dicts_train = DatasetCatalog.get("TACO_train")

#register_coco_instances("TACO_val", {}, val_dataset.annotation_file, val_dataset.image_dir)
#MetadataCatalog.get("TACO_val").set(thing_classes=val_dataset.classes)
#dataset_dicts_val = DatasetCatalog.get("TACO_val")

register_coco_instances("TACO_train", {}, train_annotation_file, img_dir)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/10 06:07:46 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/10 06:07:46 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json


In [10]:
print(dataset_dicts_val)

[{'file_name': '/content/official/batch_1/000058.jpg', 'height': 2049, 'width': 1537, 'image_id': 15, 'annotations': [{'iscrowd': 0, 'bbox': [150.0, 328.0, 568.0, 1304.0], 'category_id': 0, 'segmentation': [[249.0, 1458.0, 236.0, 1360.0, 222.0, 1247.0, 150.0, 784.0, 150.0, 747.0, 157.0, 712.0, 172.0, 675.0, 184.0, 656.0, 205.0, 632.0, 238.0, 603.0, 263.0, 589.0, 299.0, 575.0, 312.0, 570.0, 322.0, 568.0, 324.0, 567.0, 323.0, 538.0, 319.0, 497.0, 314.0, 432.0, 307.0, 362.0, 307.0, 354.0, 313.0, 346.0, 317.0, 343.0, 328.0, 338.0, 345.0, 333.0, 355.0, 330.0, 363.0, 328.0, 371.0, 334.0, 386.0, 332.0, 404.0, 330.0, 421.0, 330.0, 450.0, 332.0, 476.0, 335.0, 486.0, 338.0, 489.0, 337.0, 493.0, 335.0, 501.0, 337.0, 509.0, 341.0, 518.0, 346.0, 527.0, 351.0, 531.0, 359.0, 531.0, 369.0, 532.0, 400.0, 533.0, 429.0, 538.0, 526.0, 541.0, 566.0, 545.0, 568.0, 582.0, 580.0, 626.0, 604.0, 655.0, 627.0, 688.0, 667.0, 707.0, 704.0, 717.0, 753.0, 718.0, 806.0, 712.0, 952.0, 706.0, 1090.0, 698.0, 1222.0, 693

In [14]:
train_cfg = cfg_solov2.clone()
with open("/content/TACO-expl/taco_train_solov2.yaml", "w") as f:
  f.write(train_cfg.dump())

In [53]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/solov2_config/taco_train_solov2.yaml")
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)
print(train_cfg_loaded.DATALOADER.NUM_WORKERS)
print(train_cfg_loaded.OUTPUT_DIR)
print(train_cfg_loaded.MODEL.WEIGHTS)

8
4
/content/output/chkpt/
/content/output/chkpt/model_0001949.pth


In [14]:
%cd /content/TACO-expl/AdelaiDet/tools/
from train_net import Trainer
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

/content/TACO-expl/AdelaiDet/tools


In [15]:
%cd /content/

/content


In [52]:
trainer = Trainer(train_cfg_loaded)
trainer.build_hooks()

[05/10 06:29:47 d2.engine.defaults]: Model:
SOLOv2(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Seque

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


 None,

In [79]:
some_dict = trainer.checkpointer.resume_or_load(resume = True, path = '/content/output/chkpt/model_0001949.pth')

[05/10 07:01:43 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                              | Names in Checkpoint                                                                                  | Shapes                                          |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*           | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*           | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*           | backbone.bottom_up.res2.0.conv3.{norm.bias,norm.running_mean,

In [99]:
last_chkpt_path = '/content/output/chkpt/model_0001949.pth'
with open('/content/output/chkpt/last_checkpoint', 'w', encoding = 'ascii') as f:
  f.write(last_chkpt_path)

In [100]:
with open('/content/output/chkpt/last_checkpoint', 'r', encoding = 'ascii') as f:
  print(f.read())

/content/output/chkpt/model_0001949.pth


In [101]:
print(some_dict.keys())
#print(some_dict['optimizer'])
#print('_________')
#print(some_dict['iteration'])
#print(trainer.scheduler._step_count)
#trainer.resume_or_load(resume = True)
#print(trainer.start_iter)
#print(trainer.resume_or_load(resume = True))
trainer.checkpointer.has_checkpoint()

dict_keys(['optimizer', 'scheduler', 'iteration', 'matching_heuristics'])


True

In [ ]:
#trainer = Trainer(train_cfg_loaded)
#trainer.build_hooks()
trainer.resume_or_load(resume = True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

[05/10 07:19:26 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                              | Names in Checkpoint                                                                                  | Shapes                                          |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*           | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*           | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*           | backbone.bottom_up.res2.0.conv3.{norm.bias,norm.running_mean,

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 07:20:08 d2.utils.events]:  eta: 3:27:05  iter: 1959  total_loss: 1.25  loss_ins: 0.9572  loss_cate: 0.2914  time: 3.7122  data_time: 2.2751  lr: 0.00076578  max_mem: 8286M
[05/10 07:21:13 d2.utils.events]:  eta: 2:58:10  iter: 1979  total_loss: 1.426  loss_ins: 1.122  loss_cate: 0.2944  time: 3.3850  data_time: 1.6352  lr: 0.00076145  max_mem: 8419M
[05/10 07:22:24 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json
[05/10 07:22:24 d2.data.build]: Distribution of instances among all 10 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
|   Bottle   | 50           |  Bottle cap   | 30           |    Can     | 21           |
| Cigarette  | 42           |      Cup      | 21           |    Lid     | 8            |
|   Other    | 135          | Plastic bag.. | 77           |  Pop tab   

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 07:22:58 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0019 s/iter. Inference: 0.2404 s/iter. Eval: 4.2811 s/iter. Total: 4.5234 s/iter. ETA=0:10:28
[05/10 07:23:04 d2.evaluation.evaluator]: Inference done 12/150. Dataloading: 0.0021 s/iter. Inference: 0.2417 s/iter. Eval: 4.4006 s/iter. Total: 4.6453 s/iter. ETA=0:10:41
[05/10 07:23:15 d2.evaluation.evaluator]: Inference done 15/150. Dataloading: 0.0036 s/iter. Inference: 0.2469 s/iter. Eval: 4.1582 s/iter. Total: 4.4099 s/iter. ETA=0:09:55
[05/10 07:23:23 d2.evaluation.evaluator]: Inference done 18/150. Dataloading: 0.0033 s/iter. Inference: 0.2304 s/iter. Eval: 3.7105 s/iter. Total: 3.9452 s/iter. ETA=0:08:40
[05/10 07:23:33 d2.evaluation.evaluator]: Inference done 19/150. Dataloading: 0.0033 s/iter. Inference: 0.2293 s/iter. Eval: 4.1802 s/iter. Total: 4.4139 s/iter. ETA=0:09:38
[05/10 07:23:48 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.0033 s/iter. Inference: 0.2462 s/iter. Eval: 4.558

...

[05/10 07:33:46 d2.evaluation.evaluator]: Total inference time: 0:11:15.079947 (4.655724 s / iter per device, on 1 devices)
[05/10 07:33:46 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:30 (0.207239 s / iter per device, on 1 devices)
[05/10 07:33:47 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/10 07:33:47 d2.evaluation.coco_evaluation]: Saving results to /content/output/chkpt/inference/coco_instances_results.json
[05/10 07:33:47 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
[05/10 07:33:47 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/10 07:33:47 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.10 seconds.
[05/10 07:33:47 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[05/10 07:33:47 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.04 seconds.
 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 08:01:56 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0028 s/iter. Inference: 0.2040 s/iter. Eval: 4.3886 s/iter. Total: 4.5955 s/iter. ETA=0:10:38
[05/10 08:02:01 d2.evaluation.evaluator]: Inference done 13/150. Dataloading: 0.0027 s/iter. Inference: 0.2028 s/iter. Eval: 3.8762 s/iter. Total: 4.0821 s/iter. ETA=0:09:19
[05/10 08:02:16 d2.evaluation.evaluator]: Inference done 15/150. Dataloading: 0.0038 s/iter. Inference: 0.2079 s/iter. Eval: 4.5357 s/iter. Total: 4.7480 s/iter. ETA=0:10:40
[05/10 08:02:22 d2.evaluation.evaluator]: Inference done 18/150. Dataloading: 0.0046 s/iter. Inference: 0.2041 s/iter. Eval: 3.8998 s/iter. Total: 4.1092 s/iter. ETA=0:09:02
[05/10 08:02:27 d2.evaluation.evaluator]: Inference done 20/150. Dataloading: 0.0044 s/iter. Inference: 0.1992 s/iter. Eval: 3.7039 s/iter. Total: 3.9081 s/iter. ETA=0:08:28
[05/10 08:02:33 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.0044 s/iter. Inference: 0.1964 s/iter. Eval: 3.837

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 08:37:21 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0023 s/iter. Inference: 0.2129 s/iter. Eval: 4.2606 s/iter. Total: 4.4759 s/iter. ETA=0:10:22
[05/10 08:37:26 d2.evaluation.evaluator]: Inference done 12/150. Dataloading: 0.0025 s/iter. Inference: 0.2134 s/iter. Eval: 4.3381 s/iter. Total: 4.5550 s/iter. ETA=0:10:28
[05/10 08:37:40 d2.evaluation.evaluator]: Inference done 15/150. Dataloading: 0.0042 s/iter. Inference: 0.2162 s/iter. Eval: 4.3072 s/iter. Total: 4.5287 s/iter. ETA=0:10:11
[05/10 08:37:49 d2.evaluation.evaluator]: Inference done 19/150. Dataloading: 0.0040 s/iter. Inference: 0.2142 s/iter. Eval: 3.6855 s/iter. Total: 3.9048 s/iter. ETA=0:08:31
[05/10 08:37:59 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.0038 s/iter. Inference: 0.2106 s/iter. Eval: 3.7991 s/iter. Total: 4.0145 s/iter. ETA=0:08:37
[05/10 08:38:07 d2.evaluation.evaluator]: Inference done 22/150. Dataloading: 0.0039 s/iter. Inference: 0.2118 s/iter. Eval: 4.060

In [26]:
help(trainer)

Help on Trainer in module train_net object:

class Trainer(detectron2.engine.defaults.DefaultTrainer)
 |  Trainer(cfg)
 |  
 |  This is the same Trainer except that we rewrite the
 |  `build_train_loader`/`resume_or_load` method.
 |  
 |  Method resolution order:
 |      Trainer
 |      detectron2.engine.defaults.DefaultTrainer
 |      detectron2.engine.train_loop.TrainerBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  build_hooks(self)
 |      Replace `DetectionCheckpointer` with `AdetCheckpointer`.
 |      
 |      Build a list of default hooks, including timing, evaluation,
 |      checkpointing, lr scheduling, precise BN, writing events.
 |  
 |  resume_or_load(self, resume=True)
 |      If `resume==True` and `cfg.OUTPUT_DIR` contains the last checkpoint (defined by
 |      a `last_checkpoint` file), resume from the file. Resuming means loading all
 |      available states (eg. optimizer and scheduler) and update iteration counter
 |      from the checkpoint. ``

# Run this if you want to check how many trainable parameters we have

In [ ]:
from detectron2.modeling import build_model
model = build_model(train_cfg_loaded)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

On training set: the maximum number of instances in a single image is 90.
The average number of instances is 3.27
On validation set: max is 16 mean is 2.59
On test: max is 37 mean is 2.77